In [1]:
import torch
import numpy as np
import scipy.integrate
solver = scipy.integrate.solve_ivp
from numpy import sin, cos 
import matplotlib.pyplot as plt
import random


seed = 3
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.determinstic = True

In [2]:
#parameters
k1 = 3.0
k2 = 5.0
m1 = 1.0
m2 = 2.0
l1 = 1.0
l2 = 1.0

def func(t, state):
  dvdt = np.zeros_like(state)
  dvdt[0] = state[2]
  dvdt[1] = state[3]
  dvdt[2] = (-k1*(state[0]-l1)+k2*(state[1]-state[0]-l2))/m1
  dvdt[3] = -k2*(state[1]-state[0]-l2)/m2
  return dvdt

M = 100
tend = 5.0
t_eval = np.linspace(0,tend,M)
dt = t_eval[1]-t_eval[0]


num = 1000
x1_init = np.random.randn(num)
x2_init = np.random.randn(num)
#print(x1)
v1_init = np.random.randn(num)
v2_init = np.random.randn(num)
state = []
for i in range(num):
  s = (x1_init[i], v1_init[i], x2_init[i], v2_init[i])
  state.append(s)
#print(state)
p1 = []
v1 = []
p2 = []
v2 = []
flag = False
for i in range(num):
  sol = solver(func, [0, tend], state[i], t_eval=t_eval)
  tval = sol['t']
  dv1dt = sol['y'][0], sol['y'][1]
  dv2dt = sol['y'][2], sol['y'][3]
  p1 = dv1dt[0]
  v1 = dv1dt[1] 
  p2 = dv2dt[0]
  v2 = dv2dt[1]
  q1 = sol['y'][0]
  q2 = sol['y'][1]
  v1 = sol['y'][2]
  v2 = sol['y'][3]
  xval = np.vstack([q1, q2, v1, v2])
  dxdt = func(tval, xval)
  if flag:
    x_input = np.concatenate([x_input, xval], 1)
    x_target = np.concatenate([x_target, dxdt], 1)
  else:
    x_input = xval
    x_target = dxdt
    flag = True

x_target_max = np.abs(x_target).max(axis=1)
A = np.diag(x_target_max)
B = A

x_input = np.matmul(np.linalg.inv(B),x_input)
x_target = np.linalg.inv(A)@x_target

In [4]:
target_file = np.savetxt("target.csv", x_target.T, delimiter=',')
input_file = np.savetxt("input.csv", x_input.T, delimiter=',')

In [5]:
A_file = np.savetxt("A.csv", A, delimiter=',')
B_file = np.savetxt("B.csv", B, delimiter=',')